In [1]:
import random
import os
import numpy as np
import pandas as pd
from datetime import datetime
from autogluon.tabular import TabularPredictor

random.seed(42)
os.environ['PYTHONHASHSEED'] = str(42)
np.random.seed(42)

# ==========================================
# 1. 데이터 로드
# ==========================================

train_df = pd.read_csv('../Data/train.csv')
test_df = pd.read_csv('../Data/test.csv')
target = '임신 성공 여부'

# ==========================================
# 2. 파생변수 생성
# ==========================================

def derive_features(df):
    
    df['is_blastocyst'] = df['특정 시술 유형'].str.contains('BLASTOCYST', case=False, na=False)
    
    elderly_categories = ['만35-37세', '만38-39세', '만40-42세', '만43-44세', '만45-50세']
    df['고령 여부'] = df['시술 당시 나이'].isin(elderly_categories).astype(int)

    age_order = {'만18-34세': 1, '만35-37세': 2, '만38-39세': 3, '만40-42세': 4, '만43-44세': 5, '만45-50세': 6, '알 수 없음': 0}
    df['나이_순서'] = df['시술 당시 나이'].map(age_order)
    df['나이_제곱'] = df['나이_순서'] ** 2
    df['나이x배아'] = df['나이_순서'] * df['이식된 배아 수']
    # '나이가 적으면서 배아가 많은 그룹' vs '나이가 많으면서 배아가 적은 그룹' 강조
    df['연령별 배아 효율'] = df['이식된 배아 수'] / (df['나이_순서'] + 1e-5)

    df['배아 발달 기간'] = df['배아 이식 경과일'] - df['난자 혼합 경과일']
    df['배아 생성 효율'] = df['저장된 배아 수'] / (df['저장된 신선 난자 수'] + 1e-6)
    df['이식 비중'] = df['이식된 배아 수'] / (df['이식된 배아 수'] + df['저장된 배아 수'] + 1e-5)

    df['이식배아_구간'] = pd.cut(
    df['이식된 배아 수'].fillna(0),
    bins=[-float('inf'), 0, 2, float('inf')],
    labels=['0개', '1-2개', '3개 이상']
    )

    def clean_treatment(text): 
        text = str(text).upper()
        if 'ICSI' in text:
            return 'ICSI'
        if 'IVF' in text:
            return 'IVF'
        if 'IUI' in text:
            return 'IUI'
        return 'Other'

    df['시술유형_정제'] = df['특정 시술 유형'].apply(clean_treatment)
    
    drop_cols = ['특정 시술 유형', '시술 유형', 'DI 출산 횟수', '대리모 여부', 
                 '난자 기증자 나이', '저장된 신선 난자 수', 'PGD 시술 여부', '난자 혼합 경과일',
                 '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '불임 원인 - 자궁경부 문제', '불임 원인 - 정자 농도']
    df.drop(drop_cols, axis=1, inplace=True)
    
    return df

train_df = derive_features(train_df)
test_df = derive_features(test_df)

# ==========================================
# 3. 모델 학습 설정
# ==========================================

predictor = TabularPredictor(
    label=target, 
    eval_metric='roc_auc',
    path='ag_models_out',
).fit(
    train_data=train_df,
    time_limit=3600,
    presets='best_quality',
    ag_args_fit={'num_gpus': 1},
    num_bag_folds=5,
    num_stack_levels=3,
    refit_full=True
)

# ==========================================
# 4. 예측 (Test Data 활용) - 최종 결과를 확률로 출력 (Positive 클래스에 대한 확률만 추출)
# ==========================================

pred_probs = predictor.predict_proba(test_df)
final_probs = pred_probs.iloc[:, 1]


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.5.0
Python Version:     3.11.14
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 25.2.0: Tue Nov 18 21:09:40 PST 2025; root:xnu-12377.61.12~1/RELEASE_ARM64_T6000
CPU Count:          10
Pytorch Version:    2.9.1
CUDA Version:       CUDA is not available
GPU Count:          WARNING: Exception was raised when calculating GPU count (AssertionError)
Memory Avail:       4.15 GB / 16.00 GB (26.0%)
Disk Space Avail:   190.67 GB / 460.43 GB (41.4%)
Presets specified: ['best_quality']
Using hyperparameters preset: hyperparameters='zeroshot'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=5, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether 

In [2]:
# --- 1. 리더보드 (오름차순 정렬) ---
lb = predictor.leaderboard(silent=True)
display(lb.sort_values(by='score_val', ascending=False))

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.740018,roc_auc,22.505892,2350.820244,0.030680,7.112056,2,True,10
1,XGBoost_BAG_L1,0.738821,roc_auc,1.607768,115.689540,1.607768,115.689540,1,True,6
2,NeuralNetTorch_BAG_L1,0.737322,roc_auc,4.082671,1127.784631,4.082671,1127.784631,1,True,7
3,NeuralNetFastAI_BAG_L1,0.737040,roc_auc,2.012486,1000.515935,2.012486,1000.515935,1,True,5
4,NeuralNetTorch_r79_BAG_L1,0.734639,roc_auc,4.171800,211.437797,4.171800,211.437797,1,True,8
5,ExtraTreesEntr_BAG_L1,0.731827,roc_auc,5.295477,7.425525,5.295477,7.425525,1,True,4
6,ExtraTreesGini_BAG_L1,0.731438,roc_auc,5.337596,7.283738,5.337596,7.283738,1,True,3
7,RandomForestEntr_BAG_L1,0.729893,roc_auc,5.236917,8.703051,5.236917,8.703051,1,True,2
8,RandomForestGini_BAG_L1,0.729204,roc_auc,5.370015,8.452349,5.370015,8.452349,1,True,1
9,NeuralNetFastAI_r191_BAG_L1,0.725286,roc_auc,4.165354,82.562682,4.165354,82.562682,1,True,9


In [3]:
# --- 2. 피처 중요도 ---
fi = predictor.feature_importance(data=train_df.sample(n=min(5000, len(train_df)), random_state=42))
display(fi.sort_values(by='p_value', ascending=True).head(30))

These features in provided data are not utilized by the predictor and will be ignored: ['ID', '불임 원인 - 여성 요인']
Computing feature importance via permutation shuffling for 65 features using 5000 rows with 5 shuffle sets...
	687.09s	= Expected runtime (137.42s per shuffle set)
	353.78s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
혼합된 난자 수,0.008310,0.000189,3.177682e-08,5,0.008699,0.007922
고령 여부,0.002509,0.000079,1.200663e-07,5,0.002672,0.002345
파트너 정자와 혼합된 난자 수,0.006767,0.000237,1.811763e-07,5,0.007256,0.006279
배아 생성 효율,0.003149,0.000132,3.663121e-07,5,0.003420,0.002878
총 시술 횟수,0.005280,0.000228,4.172608e-07,5,0.005750,0.004811
수집된 신선 난자 수,0.010548,0.000514,6.727227e-07,5,0.011606,0.009490
불명확 불임 원인,0.002006,0.000105,8.884317e-07,5,0.002222,0.001790
나이_제곱,0.003484,0.000203,1.385145e-06,5,0.003903,0.003066
미세주입에서 생성된 배아 수,0.006589,0.000408,1.762981e-06,5,0.007430,0.005748
미세주입 후 저장된 배아 수,0.002159,0.000136,1.891189e-06,5,0.002440,0.001879


In [4]:
# --- 3. 제출 파일 생성 ---
# submission = pd.read_csv('../Data/sample_submission.csv')
# submission['probability'] = final_probs.values

# # 현재 시간 가져오기 (예: 0206_1031)
# now = datetime.now().strftime('%m%d_%H%M')
# file_name = f"{now}_submission.csv"
# submission.to_csv(file_name, index=False)

# print(f"학습 및 예측이 완료되었습니다. 결과가 {file_name}에 저장되었습니다.")

In [6]:
fi.to_excel('fi4.xlsx')